plan: 
with script, make a string of the following format that includes all haplotypes and also filters on both cwd version 2 and 3 (must be 'C' (common) for both):

https://www.ebi.ac.uk/ipd/imgt/hla/alleles/?query=or(and(eq(cwd.version_3.Total,%22C%22),eq(cwd.version_2,%22C%22),startsWith(name,%22A*24:02%22)),and(eq(cwd.version_3.Total,%22C%22),eq(cwd.version_2,%22C%22),startsWith(name,%22A*01:02%22)))

paste resulting url string into browser and then download the result list mapping IPD Accession (what we need for getting jsons) to allele name.
then loop over all the accessions to get json urls: https://www.ebi.ac.uk/cgi-bin/ipd/api/allele/[HLA00002]
    format: https://www.ebi.ac.uk/cgi-bin/ipd/api/allele/[accession]

In [9]:
import numpy as np
import pandas as pd
import json
from Bio.Seq import Seq
import math
import urllib.request

In [10]:
def readHLA(filename):
    """
    takes in filename of HLA locus xlsx, outputs list of most frequent alleles
    an allele is defined as frequent if it occurs in more than 1% of the population for any ethnic group 
    """
    df = pd.read_excel(filename)
    freq_alleles_list = []
    cutoff = 0.01
    for index, row in df.iterrows():
        # print(row["A"])
        if max(row[1], row[3], row[5], row[7], row[9], row[11], row[13], row[15], row[17], row[19], row[21], row[23], row[25], row[27], row[29], row[31], row[33], row[35], row[37], row[39], row[41], row[43], row[45], row[47], row[49], row[51]) >= cutoff:
            # freq_alleles_list.append(row[0])
            if row[0][-1] == "g":
                freq_alleles_list.append(row[0][:-1])
            else:
                freq_alleles_list.append(row[0])
    return freq_alleles_list

In [11]:
freq_alleles = {}
freq_alleles["A"] = readHLA("./HLA_allele_freq_sheets/A.xlsx")
freq_alleles["B"] = readHLA("./HLA_allele_freq_sheets/B.xlsx")
freq_alleles["C"] = readHLA("./HLA_allele_freq_sheets/C.xlsx")
freq_alleles["DQB1"] = readHLA("./HLA_allele_freq_sheets/DQB1.xlsx")
freq_alleles["DRB1"] = readHLA("./HLA_allele_freq_sheets/DRB1.xlsx")
# freq_alleles is a dict mapping strings of gene (eg. "A", "B") to list of common haplotype names (strings)

/var/folders/sm/2bd25dyd1bx5nl6h9b6j4z7c0000gn/T/ipykernel_14161/386535371.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if max(row[1], row[3], row[5], row[7], row[9], row[11], row[13], row[15], row[17], row[19], row[21], row[23], row[25], row[27], row[29], row[31], row[33], row[35], row[37], row[39], row[41], row[43], row[45], row[47], row[49], row[51]) >= cutoff:
/var/folders/sm/2bd25dyd1bx5nl6h9b6j4z7c0000gn/T/ipykernel_14161/386535371.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if row[0][-1] == "g":
/var/folders/sm/2bd25dyd1bx5nl6h9b6j4z7c0000gn/T/ipykernel_14161/386535371.py:16: FutureWarning: Se

In [12]:
freq_alleles

{'A': ['A*30:04',
  'A*24:03',
  'A*24:02',
  'A*11:01',
  'A*34:02',
  'A*32:01',
  'A*01:01',
  'A*03:02',
  'A*68:01',
  'A*23:01',
  'A*11:02',
  'A*74:01',
  'A*02:05',
  'A*26:01',
  'A*30:01',
  'A*34:01',
  'A*26:03',
  'A*66:01',
  'A*03:01',
  'A*02:11',
  'A*02:02',
  'A*02:03',
  'A*26:02',
  'A*33:01',
  'A*30:02',
  'A*24:07',
  'A*29:02',
  'A*25:01',
  'A*68:02',
  'A*33:03',
  'A*36:01',
  'A*29:01',
  'A*02:01',
  'A*02:17',
  'A*31:01',
  'A*68:03',
  'A*02:06',
  'A*02:07'],
 'B': ['B*54:01',
  'B*55:01',
  'B*59:01',
  'B*44:02',
  'B*81:01',
  'B*13:02',
  'B*07:02',
  'B*07:05',
  'B*50:01',
  'B*35:12',
  'B*51:06',
  'B*13:01',
  'B*41:01',
  'B*15:11',
  'B*39:06',
  'B*57:01',
  'B*44:03',
  'B*51:02',
  'B*15:35',
  'B*56:01',
  'B*15:16',
  'B*57:03',
  'B*27:04',
  'B*40:10',
  'B*35:01',
  'B*27:05',
  'B*58:02',
  'B*58:01',
  'B*42:01',
  'B*15:01',
  'B*15:15',
  'B*15:06',
  'B*39:10',
  'B*15:03',
  'B*14:02',
  'B*49:01',
  'B*35:02',
  'B*51:01',
 

#### HLA-A

In [13]:
### overfilters 
# prefix = "https://www.ebi.ac.uk/ipd/imgt/hla/alleles/?query=or("
# a_str = str()
# for allele_name in freq_alleles["A"]:
#     #print(allele_name)
#     s = f",and(eq(cwd.version_3.Total,%22C%22),eq(cwd.version_2,%22C%22),startsWith(name,%22{allele_name}%22))"
#     a_str+=s
# a_str = a_str[1:]

# a_str = prefix+a_str+")"
# a_str

In [14]:
## only using CWD version 2 to filter seems to work well 
prefix = "https://www.ebi.ac.uk/ipd/imgt/hla/alleles/?query=or("
a_str = str()
for allele_name in freq_alleles["A"]:
    #print(allele_name)
    s = f",and(eq(cwd.version_2,%22C%22),startsWith(name,%22{allele_name}%22))"
    a_str+=s
a_str = a_str[1:]

a_str = prefix+a_str+")"
a_str

'https://www.ebi.ac.uk/ipd/imgt/hla/alleles/?query=or(and(eq(cwd.version_2,%22C%22),startsWith(name,%22A*30:04%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22A*24:03%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22A*24:02%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22A*11:01%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22A*34:02%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22A*32:01%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22A*01:01%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22A*03:02%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22A*68:01%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22A*23:01%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22A*11:02%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22A*74:01%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22A*02:05%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22A*26:01%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22A*30:01%22)),and(eq(cwd.versi

In [27]:
n_to_a = pd.read_csv("HLA_name_to_dbAccession/HLA_A_name_to_accession.txt", sep="\t")
n_to_a 
name_to_accession = dict(zip(n_to_a['name'], n_to_a['accession']))
name_to_accession

{'A*30:04:01:01': 'HLA00092',
 'A*24:03:01:01': 'HLA00053',
 'A*24:02:01:01': 'HLA00050',
 'A*11:01:01:01': 'HLA00043',
 'A*34:02:01:01': 'HLA00109',
 'A*32:01:01:01': 'HLA00101',
 'A*01:01:01:01': 'HLA00001',
 'A*03:02:01:01': 'HLA00040',
 'A*68:01:01:01': 'HLA00115',
 'A*68:01:02:01': 'HLA00116',
 'A*23:01:01:01': 'HLA00048',
 'A*11:02:01:01': 'HLA00044',
 'A*74:01:01:01': 'HLA00127',
 'A*02:05:01:01': 'HLA00010',
 'A*26:01:01:01': 'HLA00073',
 'A*30:01:01:01': 'HLA00089',
 'A*34:01:01:01': 'HLA00108',
 'A*26:03:01:01': 'HLA00075',
 'A*66:01:01:01': 'HLA00112',
 'A*03:01:01:01': 'HLA00037',
 'A*02:11:01:01': 'HLA00016',
 'A*02:02:01:01': 'HLA00007',
 'A*02:03:01:01': 'HLA00008',
 'A*33:01:01:01': 'HLA00104',
 'A*30:02:01:01': 'HLA00090',
 'A*24:07:01:01': 'HLA00057',
 'A*29:02:01:01': 'HLA00086',
 'A*25:01:01:01': 'HLA00071',
 'A*68:02:01:01': 'HLA00117',
 'A*33:03:01:01': 'HLA00106',
 'A*36:01:01:01': 'HLA00110',
 'A*29:01:01:01': 'HLA00085',
 'A*02:01:01:01': 'HLA00005',
 'A*02:01:

In [25]:
a_out = pd.read_csv("HLA_A_name_to_accession.txt", sep="\t")
a_out
a_out['name']
not_included = []
for a_allele in freq_alleles["A"]:
    # print(b_allele)
    if a_out['name'].str.contains(a_allele).any() == False:
        # print(b_allele)
        not_included.append(a_allele)

not_included

FileNotFoundError: [Errno 2] No such file or directory: 'HLA_A_name_to_accession.txt'

#### HLA-B

In [ ]:
len(freq_alleles["B"])
# freq_alleles["B"]

74

In [ ]:
## overfilters 
prefix = "https://www.ebi.ac.uk/ipd/imgt/hla/alleles/?query=or("
b_str = str()
for allele_name in freq_alleles["B"]:
    #print(allele_name)
    s = f",and(eq(cwd.version_3.Total,%22C%22),startsWith(name,%22{allele_name}%22))"
    b_str+=s
b_str = b_str[1:]

b_str = prefix+b_str+")"
b_str


# ## only using CWD version 2 to filter seems to work well 
# prefix = "https://www.ebi.ac.uk/ipd/imgt/hla/alleles/?query=or("
# b_str = str()
# for allele_name in freq_alleles["B"]:
#     #print(allele_name)
#     s = f",and(eq(cwd.version_2,%22C%22),startsWith(name,%22{allele_name}%22))"
#     b_str+=s
# b_str = b_str[1:]

# b_str = prefix+b_str+")"
# b_str

#### PASTE THE RESULTING LINK INTO BROWSER TO GET THE OUTPUTS !! DOWNLOAD FROM BROWSER 

'https://www.ebi.ac.uk/ipd/imgt/hla/alleles/?query=or(and(eq(cwd.version_3.Total,%22C%22),startsWith(name,%22B*54:01%22)),and(eq(cwd.version_3.Total,%22C%22),startsWith(name,%22B*55:01%22)),and(eq(cwd.version_3.Total,%22C%22),startsWith(name,%22B*59:01%22)),and(eq(cwd.version_3.Total,%22C%22),startsWith(name,%22B*44:02%22)),and(eq(cwd.version_3.Total,%22C%22),startsWith(name,%22B*81:01%22)),and(eq(cwd.version_3.Total,%22C%22),startsWith(name,%22B*13:02%22)),and(eq(cwd.version_3.Total,%22C%22),startsWith(name,%22B*07:02%22)),and(eq(cwd.version_3.Total,%22C%22),startsWith(name,%22B*07:05%22)),and(eq(cwd.version_3.Total,%22C%22),startsWith(name,%22B*50:01%22)),and(eq(cwd.version_3.Total,%22C%22),startsWith(name,%22B*35:12%22)),and(eq(cwd.version_3.Total,%22C%22),startsWith(name,%22B*51:06%22)),and(eq(cwd.version_3.Total,%22C%22),startsWith(name,%22B*13:01%22)),and(eq(cwd.version_3.Total,%22C%22),startsWith(name,%22B*41:01%22)),and(eq(cwd.version_3.Total,%22C%22),startsWith(name,%22B*15:11

In [ ]:
# checking which alleles were included from automatic searching and filtering
# manually search and get remaining alleles
b_out = pd.read_csv("HLA_B_rawOut.txt", sep="\t")
b_out 
b_out['name']
not_included = []
for b_allele in freq_alleles["B"]:
    # print(b_allele)
    if b_out['name'].str.contains(b_allele).any() == False:
        # print(b_allele)
        not_included.append(b_allele)

not_included

name_to_accession = dict(zip(b_out['name'], b_out['accession']))
name_to_accession

['B*40:10', 'B*15:06', 'B*56:02', 'B*56:04', 'B*15:27']

### HLA-C

In [ ]:
len(freq_alleles["C"])
# freq_alleles["B"]

32

In [ ]:
## only using CWD version 2 to filter seems to work well 
prefix = "https://www.ebi.ac.uk/ipd/imgt/hla/alleles/?query=or("
c_str = str()
for allele_name in freq_alleles["C"]:
    #print(allele_name)
    s = f",and(eq(cwd.version_2,%22C%22),startsWith(name,%22{allele_name}%22))"
    c_str+=s
c_str = c_str[1:]

c_str = prefix+c_str+")"
c_str

'https://www.ebi.ac.uk/ipd/imgt/hla/alleles/?query=or(and(eq(cwd.version_2,%22C%22),startsWith(name,%22C*07:02%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22C*14:02%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22C*07:04%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22C*08:04%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22C*08:01%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22C*15:02%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22C*03:04%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22C*08:06%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22C*12:04%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22C*03:02%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22C*17:01%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22C*08:03%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22C*04:01%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22C*07:01%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22C*16:01%22)),and(eq(cwd.versi

In [ ]:
# checking which alleles were included from automatic searching and filtering
# manually search and get remaining alleles
c_out = pd.read_csv("HLA_C_rawOut.txt", sep="\t")
c_out 

c_not_included = []
for c_allele in freq_alleles["C"]:
    # print(b_allele)
    if c_out['name'].str.contains(c_allele).any() == False:
        # print(b_allele)
        c_not_included.append(c_allele)

c_not_included

# name_to_accession = dict(zip(b_out['name'], b_out['accession']))
# name_to_accession

['C*08:06', 'C*17:01', 'C*04:06']

### HLA-DQB1

In [18]:
## only using CWD version 2 to filter seems to work well 
prefix = "https://www.ebi.ac.uk/ipd/imgt/hla/alleles/?query=or("
dqb1_str = str()
for allele_name in freq_alleles["DQB1"]:
    #print(allele_name)
    s = f",and(eq(cwd.version_2,%22C%22),startsWith(name,%22{allele_name}%22))"
    dqb1_str+=s
dqb1_str
dqb1_str = dqb1_str[1:]

dqb1_str = prefix+dqb1_str+")"
dqb1_str

'https://www.ebi.ac.uk/ipd/imgt/hla/alleles/?query=or(and(eq(cwd.version_2,%22C%22),startsWith(name,%22DQB1*03:02%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22DQB1*02:01%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22DQB1*05:02%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22DQB1*03:01%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22DQB1*05:03%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22DQB1*06:04%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22DQB1*04:02%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22DQB1*06:09%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22DQB1*06:02%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22DQB1*03:03%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22DQB1*04:01%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22DQB1*06:01%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22DQB1*05:01%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22DQB1*06:03%22)))'

In [26]:
n_to_a = pd.read_csv("HLA_name_to_dbAccession/HLA_DQB1_name_to_accession.txt", sep="\t")
n_to_a 
name_to_accession = dict(zip(n_to_a['name'], n_to_a['accession']))
name_to_accession

{'DQB1*03:02:01:01': 'HLA00627',
 'DQB1*02:01:01:01': 'HLA00622',
 'DQB1*05:02:01:01': 'HLA00639',
 'DQB1*03:01:01:01': 'HLA00625',
 'DQB1*05:03:01:01': 'HLA00640',
 'DQB1*06:04:01:01': 'HLA00648',
 'DQB1*04:02:01:01': 'HLA00637',
 'DQB1*06:09:01:01': 'HLA00654',
 'DQB1*06:02:01:01': 'HLA00646',
 'DQB1*03:03:02:01': 'HLA00629',
 'DQB1*04:01:01:01': 'HLA00636',
 'DQB1*06:01:01:01': 'HLA00643',
 'DQB1*05:01:01:01': 'HLA00638',
 'DQB1*06:03:01:01': 'HLA00647'}

In [37]:
dqb1_out = pd.read_csv("HLA_name_to_dbAccession/HLA_DQB1_name_to_accession.txt", sep="\t")

# print(dqb1_out['name'].str.contains('DQB1*03:02', regex=False))
not_included = []
for dqb1_allele in freq_alleles["DQB1"]:
    print(dqb1_allele)
    if dqb1_out['name'].str.contains(dqb1_allele, regex=False).any() == False:
        print(dqb1_allele)
        not_included.append(dqb1_allele)

not_included

DQB1*03:02
DQB1*02:01
DQB1*05:02
DQB1*03:01
DQB1*05:03
DQB1*06:04
DQB1*04:02
DQB1*06:09
DQB1*06:02
DQB1*03:03
DQB1*04:01
DQB1*06:01
DQB1*05:01
DQB1*06:03


[]

### HLA-DRB1

In [39]:
## only using CWD version 2 to filter seems to work well 
prefix = "https://www.ebi.ac.uk/ipd/imgt/hla/alleles/?query=or("
drb1_str = str()
for allele_name in freq_alleles["DRB1"]:
    #print(allele_name)
    s = f",and(eq(cwd.version_2,%22C%22),startsWith(name,%22{allele_name}%22))"
    drb1_str+=s
drb1_str
drb1_str = drb1_str[1:]

drb1_str = prefix+drb1_str+")"
drb1_str

'https://www.ebi.ac.uk/ipd/imgt/hla/alleles/?query=or(and(eq(cwd.version_2,%22C%22),startsWith(name,%22DRB1*14:05%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22DRB1*14:04%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22DRB1*15:02%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22DRB1*04:05%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22DRB1*08:02%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22DRB1*04:07%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22DRB1*15:06%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22DRB1*13:02%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22DRB1*09:01%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22DRB1*08:04%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22DRB1*11:04%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22DRB1*14:08%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22DRB1*04:06%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22DRB1*15:03%22)),and(eq(cwd.version_2,%22C%22),starts

In [45]:
n_to_a = pd.read_csv("HLA_name_to_dbAccession/HLA_DRB1_name_to_accession.txt", sep="\t")
n_to_a 
name_to_accession = dict(zip(n_to_a['name'], n_to_a['accession']))
name_to_accession
len(name_to_accession)

49

In [47]:
drb1_out = pd.read_csv("HLA_name_to_dbAccession/HLA_DRB1_name_to_accession.txt", sep="\t")

# print(dqb1_out['name'].str.contains('DQB1*03:02', regex=False))
not_included = []
for drb1_allele in freq_alleles["DRB1"]:
    print(drb1_allele)
    if drb1_out['name'].str.contains(drb1_allele, regex=False).any() == False:
        print(drb1_allele)
        not_included.append(drb1_allele)

print("not included", not_included)

DRB1*14:05
DRB1*14:04
DRB1*15:02
DRB1*04:05
DRB1*08:02
DRB1*04:07
DRB1*15:06
DRB1*13:02
DRB1*09:01
DRB1*08:04
DRB1*11:04
DRB1*14:08
DRB1*04:06
DRB1*15:03
DRB1*13:03
DRB1*04:02
DRB1*12:01
DRB1*04:01
DRB1*01:02
DRB1*11:01
DRB1*14:02
DRB1*15:01
DRB1*03:02
DRB1*14:01
DRB1*03:01
DRB1*01:01
DRB1*12:02
DRB1*14:03
DRB1*13:04
DRB1*08:01
DRB1*04:04
DRB1*14:06
DRB1*13:01
DRB1*13:12
DRB1*11:02
DRB1*01:03
DRB1*16:01
DRB1*04:11
DRB1*04:10
DRB1*07:01
DRB1*10:01
DRB1*04:03
DRB1*16:02
DRB1*08:03
not included []
